In [1]:
import lib_non_local_games as nlg
import numpy as np
import cvxpy as cp
import math
import functools as fc
from operator import mul

In [2]:
if not 'MOSEK' in cp.installed_solvers():
    raise RuntimeError('Please install MOSEK before running this main file.')

### Define the dimensions

In [3]:
dimA1 = 2
dimA2 = 2
dimQ1 = 2
dimQ2 = 2
dimT = 2
dimS = 2 # always same as dimT

### Inputs
V(a1,a2,q1,q2): CHSH rule function, and $\pi(q1,q2)$: the uniform distribution

In [4]:
Rule_function = nlg.CHSH_rule_function_A1Q1A2Q2

probQ1 = (1/2,1/2)
probQ2 = (1/2,1/2)

## Linear programming (|T|=1)

In this case, the structure of $p(a1,a2,q1,q2)$ is bipartite, so we can extend only one system.

In [5]:
def CHSH_classical_T_1(level=1):
    # Compute the dimension of the new variable
    subs_A1Q1 = (dimA1, dimQ1)
    subs_A2Q2 = (dimA2, dimQ2)
    subs_A1Q1A2Q2 = (dimA1, dimQ1, dimA2, dimQ2)
    dim_A1Q1 = fc.reduce(mul, subs_A1Q1, 1)
    dim_A2Q2 = fc.reduce(mul, subs_A2Q2, 1)
    dim_tot = dim_A1Q1*dim_A2Q2**level
    
    # Create the indices lists we need
    indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
    indices_A1Q1 = nlg.indices_list(subs_A1Q1)
    indices_A2Q2 = nlg.indices_list(subs_A2Q2)
    indices_A2Q2_extended = nlg.indices_list(subs_A2Q2*level)
    if level != 1:
        indices_A2Q2_ext_butone = nlg.indices_list(subs_A2Q2*(level-1))
    else:
        indices_A2Q2_ext_butone = []
    
    # Bit string to integer function for extended distribution
    BtI_ext = lambda seq : nlg.seqtoint(seq, subs_A1Q1+subs_A2Q2*level)
    BtI = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)
    
    ###################OPTIMISATION###################
    ## VARIABLES
    # Create the classical variable (with non-negative entries)
    classical_prob = cp.Variable(dim_tot,nonneg=True)
    
    
    ## OBJECTIVE FUNCTION
    # The rule function is
    object_function = 0
    
    # The object function is
    for index_A1Q1A2Q2 in indices_A1Q1A2Q2:
        if level != 1:
            indices_a1q1a2q2_ext = nlg.fuse_arrays(np.array([index_A1Q1A2Q2]),indices_A2Q2_ext_butone)
            object_function += Rule_function(*index_A1Q1A2Q2) * sum([classical_prob[BtI_ext(i)] for i in indices_a1q1a2q2_ext])
        else:
            object_function += Rule_function(*index_A1Q1A2Q2) * classical_prob[BtI(index_A1Q1A2Q2)]
    
    
    ## CONSTRAINTS
    constraints = []
    
    # Classical constraints for the given level of the hierarchy
    # 1) The distribution is a proper probability distribution (elements adds up to 1)
    constraints.append( cp.sum(classical_prob) - 1 == 0 )
    
    # 2) Non-signalling A
    for q1 in range(dimQ1):
        for index_A2Q2_ext in indices_A2Q2_extended:
            indices_A1q1a2q2_ext = [np.append(np.array([a,q1]),index_A2Q2_ext) for a in range(dimA1)]
            indices_A1Q1a2q2_ext = [np.append(index_A1Q1,index_A2Q2_ext) for index_A1Q1 in indices_A1Q1]
    
            rhs = sum([classical_prob[BtI_ext(i)] for i in indices_A1q1a2q2_ext])
            lhs = probQ1[q1] * sum([classical_prob[BtI_ext(i)] for i in indices_A1Q1a2q2_ext])
    
            constraints.append( rhs - lhs == 0 )
    
    # 3) Non-signalling B
    if level != 1:
        indices_A1Q1A2Q2_ext_butone = nlg.fuse_arrays(indices_A1Q1,indices_A2Q2_ext_butone)
    else:
        indices_A1Q1A2Q2_ext_butone = indices_A1Q1
    
    for q2 in range(dimQ2):
        for index_A1Q1A2Q2_ext_butone in indices_A1Q1A2Q2_ext_butone:
            indices_a1q1A2q2_ext = [np.append(index_A1Q1A2Q2_ext_butone,np.array([a,q2])) for a in range(dimA2)]
            indices_a1q1A2Q2_ext = [np.append(index_A1Q1A2Q2_ext_butone,index_A2Q2) for index_A2Q2 in indices_A2Q2]
    
            rhs = sum([classical_prob[BtI_ext(i)] for i in indices_a1q1A2q2_ext])
            lhs = probQ2[q2] * sum([classical_prob[BtI_ext(i)] for i in indices_a1q1A2Q2_ext])
    
            constraints.append( rhs - lhs == 0 )
       
    # 4) Permutation invariance of A2Q2^level (the constraints we get here can be redundant)
    if level != 1:
        for index_A1Q1 in indices_A1Q1:
            for index_A2Q2_ext in indices_A2Q2_extended:
                index_A1Q1A2Q2_ext = np.append(index_A1Q1,index_A2Q2_ext)

                # Create all possible permutations (of the blocks A2Q2) - MAYBE THIS PART CAN BE STREAMLINED OR AT LEAST MADE CLEARER
                block_shape = (int(index_A2Q2_ext.size/2),2)
                index_A2Q2_block = np.reshape(index_A2Q2_ext,block_shape).tolist()
                permutation_list = list(nlg.permutations(index_A2Q2_block))
                permutation_array = np.array(permutation_list)
                unblock_shape = (permutation_array.shape[0],permutation_array.shape[1]*permutation_array.shape[2])
                indices_A2Q2_perm = np.unique(np.reshape(permutation_array,unblock_shape),axis=0)

                for index_A2Q2_perm in indices_A2Q2_perm:

                    if (index_A2Q2_perm == index_A2Q2_ext).all():
                        continue
                    else:
                        index_A1Q1A2Q2_perm = np.append(index_A1Q1,index_A2Q2_perm)
                        rhs = classical_prob[BtI_ext(index_A1Q1A2Q2_ext)]
                        lhs = classical_prob[BtI_ext(index_A1Q1A2Q2_perm)]
                        constraints.append( rhs - lhs == 0 )
    
    
    prob = cp.Problem(cp.Maximize(object_function), constraints)
    result = prob.solve(verbose=False,solver='MOSEK')
    
    return result

The first level (= the non-signalling value)

In [39]:
CHSH_classical_T_1(1)

1.0000000010619423

The second level

In [32]:
CHSH_classical_T_1(2)

0.750000000012367

The third level

In [33]:
CHSH_classical_T_1(3)

0.7500000006789657

The forth level

In [6]:
CHSH_classical_T_1(4)

0.7500000000057016

## SDP(1,1)

### Variables and Functions

In [5]:
# Subsystems A1 Q1 A2 Q2
subs_AQ1_AQ2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_AQ1_AQ2 = nlg.indices_list(subs_AQ1_AQ2)

# Subsystems T1 T2 S1 S2
subs_T1_T2_S1_S2 = (dimT,dimT,dimS,dimS)
dim_T1_T2_S1_S2 = fc.reduce(mul, subs_T1_T2_S1_S2, 1)

# Subsystems T2 S1 S2
subs_T2_S1_S2 = (dimT,dimS,dimS)
dim_T2_S1_S2 = fc.reduce(mul, subs_T2_S1_S2, 1)

# Subsystems Q1 A2 Q2
subs_Q1_AQ2 = (dimQ1,dimA2,dimQ2)
indices_Q1_AQ2 = nlg.indices_list(subs_Q1_AQ2)

# Subsystems A1 Q1 Q2
subs_AQ1_Q2 = (dimA1,dimQ1,dimQ2)
indices_AQ1_Q2 = nlg.indices_list(subs_AQ1_Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally entangled states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimS, dimT, dimS))
Phi_TTSS = P @ Phi_TSTS @ P

# A function which converts [a1,q1,a2,q2] to the corresponding index
StI = lambda seq : nlg.seqtoint(seq, subs_AQ1_AQ2)

### Optimisation

In [18]:
## VARIABLES 
# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(nlg.binarytoint,indices_AQ1_AQ2):
    rho_TTSS.append( cp.Variable((dim_T1_T2_S1_S2,dim_T1_T2_S1_S2),symmetric=True) )

    
## OBJECTIVE FUNCTION
# The object function is
obj_fun_components = []
for a1,q1,a2,q2 in indices_AQ1_AQ2:
    v_a1q1a2q2 = cp.Constant( int(Rule_function(a1,q1,a2,q2)) )
    variable_TTSS = rho_TTSS[StI([a1,q1,a2,q2])]
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(Phi_TTSS,variable_TTSS) ) )
    
object_function = cp.Constant(dimT**2) * sum(obj_fun_components)


## CONSTRAINTS
constraints = []

# 1) rho_T1T2TSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(StI,indices_AQ1_AQ2)]) - 1 == 0 )

# 1b) positive semidefinite matrices
for i in map(StI,indices_AQ1_AQ2):
    constraints.append( rho_TTSS[i] >> 0 )

# 2) First linear constraint
for q1,a2,q2 in indices_Q1_AQ2:
    indices_A1q1a2q2 = [StI([a1,q1,a2,q2]) for a1 in range(dimA1)]
    indices_A1Q1a2q2 = [StI([a1,qq1,a2,q2]) for a1,qq1 in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_T2_S1_S2])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 3) Second linear constraint

# The permutation matrix swaps T1 and T2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3), (1,0,2,3), subs_T1_T2_S1_S2))

for a1,q1,q2 in indices_AQ1_Q2:
    indices_a1q1A2q2 = [StI([a1,q1,a2,q2]) for a2 in range(dimA2)]
    indices_a1q1A2Q2 = [StI([a1,q1,a2,qq2]) for a2,qq2 in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )

    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_T2_S1_S2])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) PPT criterium
for i in map(StI,indices_AQ1_AQ2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_T1_T2_S1_S2,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_T1_T2_S1_S2,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_T1_T2_S1_S2,(0,1,0,0)) >> 0 )

# 5) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_A1Q1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])])
             for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_A1Q1:
    Q_row = []
    for a1p,q1p in indices_A1Q1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])])
                           for a2,q2 in indices_A2Q2])/dimQ2
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_A2Q2:
    R_row = []
    for a2p,q2p in indices_A2Q2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])])
                           for a1,q1 in indices_A1Q1])/dimQ1
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_A1Q1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])])
                  for a2,q2 in indices_A2Q2])/dimQ2)
    
for a2,q2 in indices_A2Q2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])])
                  for a1,q1 in indices_A1Q1])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )
    
# Write the problem
prob11 = cp.Problem(cp.Maximize(object_function), constraints)

### Solutions
SDP(1,1)

In [14]:
SDP11 = prob11.solve(verbose=True,solver='MOSEK')
print(SDP11)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 8193            
  Cones                  : 0               
  Scalar variables       : 2176            
  Matrix variables       : 16              
  Integer variables      : 0               

Optimizer started.
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 8193            
  Cones                  : 0               
  Scalar variables       : 2176            
  Matrix variables       : 16              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            

SDP(1,1) + PPT($T|\hat{T}|S\hat{S}$)

In [12]:
SDP11_PPT = prob11.solve(verbose=False,solver='MOSEK')
print(SDP11_PPT)

1.0000000002582694


SDP(1,1) + PPT($T|\hat{T}|S\hat{S}$) + NPA1(with the projective assumption)

In [19]:
SDP11_PPT_NPA1proj = prob11.solve(verbose=False,solver='MOSEK')
print(SDP11_PPT_NPA1proj)

0.8535533905973038


SDP(1,1) + PPT($T|\hat{T}|S\hat{S}$) + NPA1(without the projective assumption)

In [17]:
SDP11_PPT_NPA1 = prob11.solve(verbose=False,solver='MOSEK')
print(SDP11_PPT_NPA1)

0.8535533907033661


## SDP(2,1)

### Variables and Functions

In [20]:
# Subsystems A1_1 A1_2 Q1_1 Q1_2 A2 Q2
subs_AAQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQ1_AQ2 = nlg.indices_list(subs_AAQQ1_AQ2)

# Subsystems T1_1 T1_2 T2 S1 S2
subs_TT1_T2_S1_S2 = (dimT,dimT,dimT,dimS,dimS)
dim_TT1_T2_S1_S2 = fc.reduce(mul, subs_TT1_T2_S1_S2, 1)

# Subsystems T1_* T2 S1 S2
subs_T1_T2_S1_S2 = (dimT,dimT,dimS,dimS)
dim_T1_T2_S1_S2 = fc.reduce(mul, subs_T1_T2_S1_S2, 1)

# Subsystems A1_1 Q1_1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)

# Subsystems A1_1 A1_2 Q1_1 Q1_2
subs_AAQQ1 = (dimA1,dimA1,dimQ1,dimQ1)
indices_AAQQ1 = nlg.indices_list(subs_AAQQ1)

# Subsystems A1_2 Q1_1 Q1_2 A2 Q2
subs_AQQ1_AQ2 = (dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AQQ1_AQ2 = nlg.indices_list(subs_AQQ1_AQ2)

# Subsystems A1_1 A1_2 Q1_1 Q1_2 Q2
subs_AAQQ1_Q2 = (dimA1,dimA1,dimQ1,dimQ1,dimQ2)
indices_AAQQ1_Q2 = nlg.indices_list(subs_AAQQ1_Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P1 = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimS, dimT, dimS))
Phi_TTSS = P1 @ Phi_TSTS @ P1

# A function which converts [a1_1,a1_2,q1_1,q1_2,a2,q2] to the corresponding index
StI = lambda seq : nlg.seqtoint(seq, subs_AAQQ1_AQ2)
StI4 = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)

### Optimisation

In [21]:
## VARIABLES 
# The (sub-normalized) states we optimize over
rho_T1T2TSS = []
for i in map(StI,indices_AAQQ1_AQ2):
    rho_T1T2TSS.append( cp.Variable((dim_TT1_T2_S1_S2,dim_TT1_T2_S1_S2),symmetric=True) )

    
## OBJECTIVE FUNCTION
# The swap operator takes T1_1 T1_2 T2 S1 S2 to S1 T1_2 S2 T1_1 T2 & Phi_TTTSS for the objective function
F_TTTSS = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (3,1,4,0,2), subs_TT1_T2_S1_S2))
Phi_TTTSS = nlg.partial_transpose(F_TTTSS, subs_TT1_T2_S1_S2, (0,0,0,1,1))

# The object function is
obj_fun_components = []
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    v_a1q1a2q2 = cp.Constant( int(Rule_function(a1,q1,a2,q2)) )
    variable_T1T2TSS = sum([rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])] for a,q in indices_A1Q1])
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(Phi_TTTSS,variable_T1T2TSS) ) )

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)


## CONSTRAINTS
constraints = []

# 1) rho_T1T2TSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_T1T2TSS[i]) for i in map(StI,indices_AAQQ1_AQ2)]) - 1 == 0 )

# 1b) positive semidefinite matrices
for i in map(StI,indices_AAQQ1_AQ2):
    constraints.append( rho_T1T2TSS[i] >> 0 )
    
# 2) Permutation invariance over A1_1 Q1_1 T1 and A1_2 Q1_2 T2

# The permutation matrix swaps T1_1 and T1_2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (1,0,2,3,4), subs_TT1_T2_S1_S2))

for a1,a,q1,q,a2,q2 in indices_AAQQ1_AQ2:
    lhs = rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])]
    
    rhs_variable = rho_T1T2TSS[StI([a,a1,q,q1,a2,q2])]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)
    
    constraints.append( lhs - rhs == 0 )

# 3) First linear constraint
for a,q1,q,a2,q2 in indices_AQQ1_AQ2:
    indices_A1aq1qa2q2 = [StI([a1,a,q1,q,a2,q2]) for a1 in range(dimA1)]
    indices_A1aQ1qa2q2 = [StI([a1,a,qq1,q,a2,q2]) for a1,qq1 in indices_A1Q1]
    
    lhs = sum([rho_T1T2TSS[i] for i in indices_A1aq1qa2q2])
    
    rhs_variable = sum([rho_T1T2TSS[i] for i in indices_A1aQ1qa2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_T1_T2_S1_S2])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint

# The permutation matrix swaps T1_1 and T2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (2,1,0,3,4), subs_TT1_T2_S1_S2))

for a1,a,q1,q,q2 in indices_AAQQ1_Q2:
    indices_a1aq1qA2q2 = [StI([a1,a,q1,q,a2,q2]) for a2 in range(dimA2)]
    indices_a1aq1qA2Q2 = [StI([a1,a,q1,q,a2,qq2]) for a2,qq2 in indices_A2Q2]
    
    lhs_variable = sum([rho_T1T2TSS[i] for i in indices_a1aq1qA2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_T1T2TSS[i] for i in indices_a1aq1qA2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_T1_T2_S1_S2])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 5) PPT criterium
for i in map(StI,indices_AAQQ1_AQ2):
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(0,0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(1,1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(0,0,1,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(1,0,0,0,0)) >> 0 ) # More PPT
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(0,1,0,0,0)) >> 0 ) # More PPT
    
# 6) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_A1Q1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTTSS@sum([rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])] for a,q in indices_A1Q1]))
             for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_A1Q1:
    Q_row = []
    for a1p,q1p in indices_A1Q1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTSS@rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])])
                           for a,q,a2,q2 in indices_A1Q1A2Q2])/dimQ2
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_A2Q2:
    R_row = []
    for a2p,q2p in indices_A2Q2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTSS@rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])])
                           for a1,a,q1,q in indices_AAQQ1])/dimQ1
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_A1Q1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTSS@rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])])
                  for a,q,a2,q2 in indices_A1Q1A2Q2])/dimQ2)
    
for a2,q2 in indices_A2Q2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTSS@rho_T1T2TSS[StI([a1,a,q1,q,a2,q2])])
                  for a1,a,q1,q in indices_AAQQ1])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )
    
# Write the problem
prob21 = cp.Problem(cp.Maximize(object_function), constraints)

SDP(2,1)

In [23]:
SDP21 = prob21.solve(verbose=False, solver='MOSEK')
print(SDP21)

0.8943379085701879


SDP(2,1) + PPT($T_1T_2|\hat{T}|S\hat{S}$)

In [26]:
SDP21_PPT = prob21.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT)

0.8750001416919456


SDP(2,1) + PPT($T_1T_2|\hat{T}|S\hat{S}$) + more PPT : it seems like more PPT doesn't help

In [22]:
SDP21_PPTmore_NPA1 = prob21.solve(verbose=False, solver='MOSEK')
print(SDP21_PPTmore_NPA1)

0.8750000014889912


SDP(2,1) + PPT($T_1T_2|\hat{T}|S\hat{S}$) + NPA1(with the projective assumption)

In [29]:
SDP21_PPT_NPA1proj = prob21.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT_NPA1proj)

0.8535533908073588


SDP(2,1) + PPT($T_1T_2|\hat{T}|S\hat{S}$) + NPA1(without the projective assumption)

In [31]:
SDP21_PPT_NPA1 = prob21.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT_NPA1)

0.8535533907509064


Up to this point, I could run the codes with my laptop (8Gb RAM).

## SDP(2,2)

### Variables and Functions

In [32]:
# Subsystems A1_1 A1_2 Q1_1 Q1_2 A2_1 A2_2 Q2_1 Q2_2
subs_AAQQ1_AAQQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimA2,dimQ2,dimQ2)
indices_AAQQ1_AAQQ2 = nlg.indices_list(subs_AAQQ1_AAQQ2)

# Subsystems A1 Q1 A2 Q2
subs_AQ1_AQ2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_AQ1_AQ2 = nlg.indices_list(subs_AQ1_AQ2)

# Subsystems T1_1 T1_2 T2_1 T2_2 S1 S2
subs_TT1_TT2_SS = (dimT,dimT,dimT,dimT,dimS,dimS)
dim_TT1_TT2_SS = fc.reduce(mul, subs_TT1_TT2_SS, 1)

# Subsystems T*_1 T1_2 T2_2 S1 S2
subs_TTT_SS = (dimT,dimT,dimT,dimS,dimS)
dim_TTT_SS = fc.reduce(mul, subs_TTT_SS, 1)

# Subsystems A1_2 Q1_1 Q1_2 A2_1 A2_2 Q2_1 Q2_2
subs_AQQ1_AAQQ2 = (dimA1,dimQ1,dimQ1,dimA2,dimA2,dimQ2,dimQ2)
indices_AQQ1_AAQQ2 = nlg.indices_list(subs_AQQ1_AAQQ2)

# Subsystems A1_2 Q1_2 A2_1 A2_2 Q2_1 Q2_2
subs_AQ1_AAQQ2 = (dimA1,dimQ1,dimA2,dimA2,dimQ2,dimQ2)
indices_AQ1_AAQQ2 = nlg.indices_list(subs_AQ1_AAQQ2)

# Subsystems A1 Q1
subs_AQ1 = (dimA1,dimQ1)
indices_AQ1 = nlg.indices_list(subs_AQ1)

# Subsystems A1_1 A1_2 Q1_1 Q1_2 A2_2 Q2_1 Q2_2
subs_AAQQ1_AQQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2,dimQ2)
indices_AAQQ1_AQQ2 = nlg.indices_list(subs_AAQQ1_AQQ2)

# Subsystems A1_1 A1_2 Q1_1 Q1_2 A2_2 Q2_2
subs_AAQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQ1_AQ2 = nlg.indices_list(subs_AAQQ1_AQ2)

# Subsystems A2 Q2
subs_AQ2 = (dimA2,dimQ2)
indices_AQ2 = nlg.indices_list(subs_AQ2)

# Subsystems T*_1 T*_2
subs_TT = (dimT,dimT)
dim_TT = fc.reduce(mul, subs_TT, 1)

# Subsystems T1_1 T2_1 S1 S2
subs_TT_SS = (dimT,dimT,dimS,dimS)
dim_TT_SS = fc.reduce(mul, subs_TT_SS, 1)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

# A function which converts [a1_1,a1_2,q1_1,q1_2,a2_1,a2_2,q2_1,q2_2] to the corresponding index
StI = lambda seq : nlg.seqtoint(seq, subs_AAQQ1_AAQQ2)

In [34]:
# VARIABLES 
# The (sub-normalized) states we optimize over
rho_TTTTSS = []
shape_TTTTSS = (dim_TT1_TT2_SS, dim_TT1_TT2_SS)

for i in map(StI,indices_AAQQ1_AAQQ2):
    rho_TTTTSS.append( cp.Variable(shape_TTTTSS,symmetric=True) )

    
## OBJECTIVE FUNCTION
# The swap operator takes T1_1 T1_2 T2_1 T2_2 S1 S2 to S1 T1_2 S2 T2_2 T1_1 T2_1 & Phi_TTTTSS for the objective function
F_TTTTSS = cp.Constant(nlg.permutation_matrix((0,1,2,3,4,5), (4,1,5,3,0,2), subs_TT1_TT2_SS))
Phi_TTTTSS = nlg.partial_transpose(F_TTTTSS, subs_TT1_TT2_SS, (0,0,0,0,1,1))

# The object function is
obj_fun_components = []

for a1,q1,a2,q2 in indices_AQ1_AQ2:
    v_a1q1a2q2 = int( Rule_function(a1,q1,a2,q2) )
    variable_TTTTSS = sum([rho_TTTTSS[StI([a1,aa1,q1,qq1,a2,aa2,q2,qq2])] for aa1,qq1,aa2,qq2 in indices_AQ1_AQ2])
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(Phi_TTTTSS,variable_TTTTSS) ) )
    
object_function = cp.Constant(dimT**2) * sum(obj_fun_components)


## CONSTRAINTS
constraints = []

# 1) rho_TTTTSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
trace_rho = sum([cp.trace(rho_TTTTSS[i]) for i in map(StI,indices_AAQQ1_AAQQ2)])
constraints.append( trace_rho - 1 == 0 )

# 1b) positive semidefinite matrices
for i in map(StI,indices_AAQQ1_AAQQ2):
    constraints.append( rho_TTTTSS[i] >> 0 )

# 2) Permutation invariance
# 2a) over A1_1 Q1_1 T1_1 and A1_2 Q1_2 T1_2

# The permutation matrix swaps T1_1 and T1_2
P = nlg.permutation_matrix((0,1,2,3,4,5), (1,0,2,3,4,5), subs_TT1_TT2_SS)

for a1,aa1,q1,qq1,a2,aa2,q2,qq2 in indices_AAQQ1_AAQQ2:
    lhs = rho_TTTTSS[StI([a1,aa1,q1,qq1,a2,aa2,q2,qq2])]
    
    rhs_variable = rho_TTTTSS[StI([aa1,a1,qq1,q1,a2,aa2,q2,qq2])]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)
    
    constraints.append( lhs - rhs == 0 )

# 2b) over A2_1 Q2_1 T2_1 and A2_2 Q2_2 T2_2

# The permutation matrix swaps T2_1 and T2_2
P = nlg.permutation_matrix((0,1,2,3,4,5), (0,1,3,2,4,5), subs_TT1_TT2_SS)

for a1,aa1,q1,qq1,a2,aa2,q2,qq2 in indices_AAQQ1_AAQQ2:
    lhs = rho_TTTTSS[StI([a1,aa1,q1,qq1,a2,aa2,q2,qq2])]
    
    rhs_variable = rho_TTTTSS[StI([a1,aa1,q1,qq1,aa2,a2,qq2,q2])]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)
    
    constraints.append( lhs - rhs == 0 )

# 3) First linear constraint
for aa1,q1,qq1,a2,aa2,q2,qq2 in indices_AQQ1_AAQQ2:
    indices_A1aa1q1qq1a2aa2q2qq2 = [StI([a,aa1,q1,qq1,a2,aa2,q2,qq2]) for a in range(dimA1)]
    indices_A1aa1Q1qq1a2aa2q2qq2 = [StI([a,aa1,q,qq1,a2,aa2,q2,qq2]) for a,q in indices_AQ1]

    lhs = sum([rho_TTTTSS[i] for i in indices_A1aa1q1qq1a2aa2q2qq2])

    rhs_variable = sum([rho_TTTTSS[i] for i in indices_A1aa1Q1qq1a2aa2q2qq2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TTT_SS])
    rhs = probQ1[q1] * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

# 4) Second linear constraint

# The permutation matrix swaps T1_1 and T2_1
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4,5), (2,1,0,3,4,5), subs_TT1_TT2_SS))

for a1,aa1,q1,qq1,aa2,q2,qq2 in indices_AAQQ1_AQQ2:
    indices_a1aa1q1qq1A2aa2q2qq2 = [StI([a1,aa1,q1,qq1,a,aa2,q2,qq2]) for a in range(dimA2)]
    indices_a1aa1q1qq1A2aa2Q2qq2 = [StI([a1,aa1,q1,qq1,a,aa2,q,qq2]) for a,q in indices_AQ2]

    lhs_variable = sum([rho_TTTTSS[i] for i in indices_a1aa1q1qq1A2aa2q2qq2])
    lhs = cp.matmul(cp.matmul(P,lhs_variable),P)

    rhs_variable = sum([rho_TTTTSS[i] for i in indices_a1aa1q1qq1A2aa2Q2qq2])
    rhs_permuted = cp.matmul(cp.matmul(P,rhs_variable),P)
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TTT_SS])
    rhs = probQ2[q2] * cp.kron(rhoT, rhs_partial)

    constraints.append( lhs - rhs == 0 )

# 5) PPT criterium
for i in map(StI,indices_AAQQ1_AAQQ2):
    constraints.append( nlg.partial_transpose(rho_TTTTSS[i],subs_TT1_TT2_SS,(0,0,0,0,1,1)) >> 0 ) # PPT1
    constraints.append( nlg.partial_transpose(rho_TTTTSS[i],subs_TT1_TT2_SS,(1,1,0,0,0,0)) >> 0 ) # PPT2
    constraints.append( nlg.partial_transpose(rho_TTTTSS[i],subs_TT1_TT2_SS,(0,0,1,1,0,0)) >> 0 ) # PPT3

# 6) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_AQ1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTTTSS@sum([rho_TTTTSS[StI([a1,a,q1,q,a2,aa,q2,qq])] for a,q,aa,qq in indices_AQ1_AQ2]))
             for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_AQ1:
    Q_row = []
    for a1p,q1p in indices_AQ1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a,q1,q,a2,aa,q2,qq])])
                           for a,q,a2,aa,q2,qq in indices_AQ1_AAQQ2])/dimQ2
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_AQ2:
    R_row = []
    for a2p,q2p in indices_AQ2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a,q1,q,a2,aa,q2,qq])])
                           for a1,a,q1,q,aa,qq in indices_AAQQ1_AQ2])/dimQ1
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_AQ1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a,q1,q,a2,aa,q2,qq])])
                  for a,q,a2,aa,q2,qq in indices_AQ1_AAQQ2])/dimQ2)
    
for a2,q2 in indices_AQ2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a,q1,q,a2,aa,q2,qq])])
                  for a1,a,q1,q,aa,qq in indices_AAQQ1_AQ2])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )
    
## PROBLEM

# Write the problem
prob22 = cp.Problem(cp.Maximize(object_function), constraints)

SDP(2,2)

In [ ]:
SDP22 = prob22.solve(verbose=False, solver='MOSEK')

SDP22 = 0.8792120993330009

The top memory requirement was 229Gb. I ran it with the computing department computer (500Gb RAM).

In [ ]:
SDP22_PPT = prob22.solve(verbose=False, solver='MOSEK')
print(SDP22)

## SDP(3,1)
### Variables and Functions

In [ ]:
# Subsystems A1_1 A1_2 A1_3 Q1_1 Q1_2 Q1_3 A2 Q2 
subs_AAAQQQ1_AQ2 = (dimA1,dimA1,dimA1,dimQ1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAAQQQ1_AQ2 = nlg.indices_list(subs_AAAQQQ1_AQ2)

# Subsystems A1_1 Q1_1 A2_1 Q2_1
subs_AQ1_AQ2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_AQ1_AQ2 = nlg.indices_list(subs_AQ1_AQ2)

# Subsystems T1_1 T1_2 T1_3 T2 S1 S2
subs_TTT1_T2_SS = (dimT,dimT,dimT,dimT,dimS,dimS)
dim_TTT1_T2_SS = fc.reduce(mul, subs_TTT1_T2_SS, 1)

# Subsystems T1_2 T1_3 T2 S1 S2
subs_TTT_SS = (dimT,dimT,dimT,dimS,dimS)
dim_TTT_SS = fc.reduce(mul, subs_TTT_SS, 1)

# Subsystems A1_1 A1_2 A1_3 Q1_1 Q1_2 Q1_3 Q2 
subs_AAAQQQ1_Q2 = (dimA1,dimA1,dimA1,dimQ1,dimQ1,dimQ1,dimQ2)
indices_AAAQQQ1_Q2 = nlg.indices_list(subs_AAAQQQ1_Q2)

# Subsystems A1_1 A1_2 A1_3 Q1_1 Q1_2 Q1_3 
subs_AAAQQQ1 = (dimA1,dimA1,dimA1,dimQ1,dimQ1,dimQ1)
indices_AAAQQQ1 = nlg.indices_list(subs_AAAQQQ1)

# Subsystems A1_2 A1_3 Q1_1 Q1_2 Q1_3 A2 Q2
subs_AAQQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQQ1_AQ2 = nlg.indices_list(subs_AAQQQ1_AQ2)

# Subsystems A1_2 A1_3 Q1_2 Q1_3 A2 Q2
subs_AAQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQ1_AQ2 = nlg.indices_list(subs_AAQQ1_AQ2)

# Subsystems A1 Q1
subs_AQ1 = (dimA1,dimQ1)
indices_AQ1 = nlg.indices_list(subs_AQ1)

# Subsystems A2 Q2
subs_AQ2 = (dimA2,dimQ2)
indices_AQ2 = nlg.indices_list(subs_AQ2)

# Subsystems A1_1 A1_2 Q1_1 Q1_2
subs_AAQQ1 = (dimA1,dimA1,dimQ1,dimQ1)
indices_AAQQ1 = nlg.indices_list(subs_AAQQ1)

# Subsystems T*_1 T*_2
subs_TT = (dimT,dimT)
dim_TT = fc.reduce(mul, subs_TT, 1)

# Subsystems T1_1 T2_1 S1 S2
subs_TT_SS = (dimT,dimT,dimS,dimS)
dim_TT_SS = fc.reduce(mul, subs_TT_SS, 1)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

# A function which converts [a1_1,a1_2,q1_1,q1_2,a2_1,a2_2,q2_1,q2_2] to the corresponding index
StI = lambda seq : nlg.seqtoint(seq, subs_AAAQQQ1_AQ2)

### Optimisation

In [ ]:
### Variables
# The (sub-normalized) states we optimize over
rho_TTTTSS = []
shape_TTTTSS = (dim_TTT1_T2_SS, dim_TTT1_T2_SS)

for index in indices_AAAQQQ1_AQ2:
    rho_TTTTSS.append( cp.Variable(shape_TTTTSS,symmetric=True) )

    
## OBJECTIVE FUNCTION
# The swap operator takes T1_1 T1_2 T1_3 T2 S1 S2 to S1 T1_2 T1_3 S2 T1_1 T2 & Phi_TTTTSS for the objective function
F_TTTTSS = cp.Constant(nlg.permutation_matrix((0,1,2,3,4,5), (4,1,2,5,0,3), subs_TTT1_T2_SS))
Phi_TTTTSS = nlg.partial_transpose(F_TTTTSS, subs_TT1_TT2_SS, (0,0,0,0,1,1))

# The object function is
obj_fun_components = []

for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    v_a1q1a2q2 = Rule_function(a1,q1,a2,q2)
    variable_TTTTSS = sum([rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2],subs_AAAQQQ1_AQ2)] for a11,a12,q11,q12 in indices_AAQQ1])
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(Phi_TTTTSS,variable_TTTTSS) ) )
    
object_function = cp.Constant(dimT**2) * sum(obj_fun_components)


## CONSTRAINTS
constraints = []

# 1) rho_TTTTSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
trace_rho = sum([cp.trace(rho_TTTTSS[StI(index,subs_AAAQQQ1_AQ2)]) for index in indices_AAAQQQ1_AQ2])
constraints.append( trace_rho - 1 == 0 )

# 1b) positive semidefinite matrices
for index in indices_AAAQQQ1_AQ2:
    constraints.append( rho_TTTTSS[StI(index,subs_AAAQQQ1_AQ2)] >> 0 )

# 2) Permutation invariance
# 2a) over A1_1 Q1_1 T1_1 and A1_2 Q1_2 T1_2

# The permutation matrix swaps T1_1 and T1_2
P = nlg.permutation_matrix((0,1,2,3,4,5), (1,0,2,3,4,5), subs_TTT1_T2_SS)

for a1,a11,a12,q1,q11,q12,a2,q2 in indices_AAAQQQ1_AQ2:
    lhs = rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2],subs_AAAQQQ1_AQ2)]
    
    rhs_variable = rho_TTTTSS[StI([a11,a1,a12,q11,q1,q12,a2,q2],subs_AAAQQQ1_AQ2)]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)

    constraints.append( lhs - rhs == 0 )

# 2b) over A1_1 Q1_1 T1_1 and A1_3 Q1_3 T1_3

# The permutation matrix swaps T1_1 and T1_3
P = nlg.permutation_matrix((0,1,2,3,4,5), (2,1,0,3,4,5), subs_TTT1_T2_SS)

for a1,a11,a12,q1,q11,q12,a2,q2 in indices_AAAQQQ1_AQ2:
    lhs = rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2],subs_AAAQQQ1_AQ2)]
    
    rhs_variable = rho_TTTTSS[StI([a12,a11,a1,q12,q11,q1,a2,q2],subs_AAAQQQ1_AQ2)]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)

    constraints.append( lhs - rhs == 0 )

# 2c) over A1_2 Q1_2 T1_2 and A1_3 Q1_3 T1_3

# The permutation matrix swaps T1_2 and T1_3
P = nlg.permutation_matrix((0,1,2,3,4,5), (0,2,1,3,4,5), subs_TTT1_T2_SS)

for a1,a11,a12,q1,q11,q12,a2,q2 in indices_AAAQQQ1_AQ2:
    lhs = rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2],subs_AAAQQQ1_AQ2)]
    
    rhs_variable = rho_TTTTSS[StI([a1,a12,a11,q1,q12,q11,a2,q2],subs_AAAQQQ1_AQ2)]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)
    
    constraints.append( lhs - rhs == 0 )

# 3) First linear constraint
for a11,a12,q1,q11,q12,a2,q2 in indices_AAQQQ1_AQ2:
    indices_A1a11a12q1q11q12a2q2 = [StI([a,a11,a12,q1,q11,q12,a2,q2],subs_AAAQQQ1_AQ2) for a in range(dimA1)]
    indices_A1a11a12Q1q11q12a2q2 = [StI([a,a11,a12,q,q11,q12,a2,q2],subs_AAAQQQ1_AQ2) for a,q in indices_AQ1]

    lhs = sum([rho_TTTTSS[i] for i in indices_A1a11a12q1q11q12a2q2])

    rhs_variable = sum([rho_TTTTSS[i] for i in indices_A1a11a12Q1q11q12a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TTT_SS])
    rhs = probQ1[q1] * cp.kron(rhoT, rhs_partial)

    constraints.append( lhs - rhs == 0 )

# 4) Second linear constraint

# The permutation matrix swaps T(1)1 and T2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4,5), (3,1,2,0,4,5), subs_TTT1_T2_SS))

for a1,a11,a12,q1,q11,q12,q2 in indices_AAAQQQ1_Q2:
    indices_a1a11a12q1q11q12A2q2 = [StI([a1,a11,a12,q1,q11,q12,a,q2],subs_AAAQQQ1_AQ2) for a in range(dimA2)]
    indices_a1a11a12q1q11q12A2Q2 = [StI([a1,a11,a12,q1,q11,q12,a,q],subs_AAAQQQ1_AQ2) for a,q in indices_AQ2]

    lhs_variable = sum([rho_TTTTSS[i] for i in indices_a1a11a12q1q11q12A2q2])
    lhs = cp.matmul(cp.matmul(P,lhs_variable),P)

    rhs_variable = sum([rho_TTTTSS[i] for i in indices_a1a11a12q1q11q12A2Q2])
    rhs_permuted = cp.matmul(cp.matmul(P,rhs_variable),P)
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TTT_SS])
    rhs = probQ2[q2] * cp.kron(rhoT, rhs_partial)

    constraints.append( lhs - rhs == 0 )

# 5) PPT criterium
for index in indices_AAAQQQ1_AQ2:
    constraints.append( nlg.partial_transpose(rho_TTTTSS[StI(index,subs_AAAQQQ1_AQ2)],subs_TTT1_T2_SS,(0,0,0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTTTSS[StI(index,subs_AAAQQQ1_AQ2)],subs_TT1_TT2_SS,(1,1,1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTTTSS[StI(index,subs_AAAQQQ1_AQ2)],subs_TT1_TT2_SS,(0,0,0,1,0,0)) >> 0 )

# 6) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_AQ1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTTTSS@sum([rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2])] for a11,a12,q11,q12 in indices_AAQQ1]))
             for a2,q2 in indices_AQ2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_AQ1:
    Q_row = []
    for a1p,q1p in indices_AQ1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a11,q12,q1,q11,q12,a2,q2])])
                           for a11,a12,q11,q12,a2,q2 in indices_AAQQ1_AQ2])/dimQ2
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_AQ2:
    R_row = []
    for a2p,q2p in indices_AQ2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2])])
                           for a1,a11,a12,q1,q11,q12 in indices_AAAQQQ1])/dimQ1
            else:
                val = cp.Constant(0) # Assume projective measurements. Otherwise, cp.Variable(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_AQ1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2])])
                  for a11,a12,q11,q12,a2,q2 in indices_AAQQ1_AQ2])/dimQ2)
    
for a2,q2 in indices_AQ2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTTTSS@rho_TTTTSS[StI([a1,a11,a12,q1,q11,q12,a2,q2])])
                  for a1,a11,a12,q1,q11,q12 in indices_AAAQQQ1])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )

    
## PROBLEM
# Write the problem
prob31 = cp.Problem(cp.Maximize(object_function), constraints)

SDP(3,1)

In [ ]:
SDP31 = prob31.solve(verbose=False, solver='MOSEK')
print(SDP31)